In [1]:
# imports
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd
import json


/Users/andrew-misnik/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#config

min_cost = 90000
max_cost = 120000
room1 = 0
room2 = 1
room3 = 1
room4 = 1
limit = 10
overall_limit = 10
min_area = 55
max_area = 0
sort = 'creation_date_desc'

'''
пусто - по умолчанию
price_object_order - сначала дешевле
total_price_desc - сначала дороже
walking_time - по возрастанию времени до метро
area_order - по убыванию площади
creation_date_desc - сначала новые
creation_date_asc - сначала старые
'''

page_number = 1
MAIN_DIR = "/Users/andrew-misnik/Documents/cian_parser/cian_parser/"

first_sleep = 6
sleep_interval = 7

lists = {
    'Ссылка': [],
    'Цена': [],
    'Flat Score': [],
    'Underground Score': [],
    'Underground Place': [],
    'Близость к метро': [],
    'Количество комнат': [],
    'Общая площадь' : [],
    'Жилая площадь': [],
    'Площадь кухни': [],
    'Этаж': [],
    'Количество этажей': [],
    'Депозит': [],
    'Месяцы предоплаты': [],
    'Ремонт': [],
    'Апартаменты': [],
    'Лоджии': [],
    'Балконы': [],
    'Окна': [],
    'Разделенные с/у': [],
    'Совмещенные с/у': [],
    'Посудомойка': [],
    'Кондиционер': [],
    'Можно с детьми': [],
    'Можно с животными': [],
    'Обновлено': [],
    'Высота потолков': [],
    'Количество подъездов': [],
    'Количество лифтов в доме': [],
    'Количество квартир в доме': [],
}

station_scores = {'Пушкинская':31.7113,'Театральная':32.0624,'Чеховская':32.067,'Тверская':32.1316,'Кузнецкий Мост':32.2286,'Третьяковская':32.2402,'Арбатская':32.2425,'Китай-город':32.2494,'Боровицкая':32.3764,'Площадь Революции':32.455,'Таганская':32.4642,'Киевская':32.5127,'Новокузнецкая':32.5404,'Добрынинская':32.6259,'Краснопресненская':32.649,'Октябрьская':32.6674,'Лубянка':32.6744,'Баррикадная':32.7113,'Тургеневская':32.7275,'Охотный Ряд':32.7575,'Павелецкая':32.7806,'Курская':32.7898,'Парк культуры':32.8152,'Библиотека имени Ленина':32.8568,'Чистые пруды':32.903,'Цветной бульвар':32.9515,'Сретенский бульвар':33.1848,'Марксистская':33.2725,'Белорусская':33.3926,'Новослободская':33.5173,'Смоленская':33.7737,'Маяковская':33.8037,'Менделеевская':33.8176,'Александровский сад':33.8199,'Трубная':33.8614,'Серпуховская':33.9122,'Комсомольская':33.9376,'Проспект Мира':34.0069,'Чкаловская':34.0508,'Полянка':34.2309,'Пролетарская':34.4111,'Савёловская':34.4319,'Шаболовская':34.515,'Сухаревская':34.6005,'Красные Ворота':34.6582,'Кропоткинская':34.6882,'Площадь Ильича':34.7044,'Римская':34.9861,'Улица 1905 года':35.0831,'Достоевская':35.3349,'Марьина Роща':35.515,'Бауманская':35.5612,'Ленинский проспект':35.7806,'Рижская':35.8406,'Динамо':35.8614,'Дмитровская':35.9538,'Беговая':35.9931,'Петровский парк':36.1224,'Полежаевская':36.1732,'Крестьянская Застава':36.2564,'Электрозаводская':36.3072,'Хорошёвская':36.321,'Волгоградский проспект':36.4619,'Автозаводская':36.5427,'Тульская':36.6097,'Парк Победы':36.6605,'Фрунзенская':36.8106,'Сокольники':36.836,'Красносельская':36.8845,'Авиамоторная':37.0323,'Площадь Гагарина':37.2979,'Тимирязевская':37.4249,'ЦСКА':37.4365,'Спортивная':37.485,'Дубровка':37.7644,'Алексеевская':37.836,'Текстильщики':37.8661,'Лужники':37.9861,'Кутузовская':38.067,'Технопарк':38.1293,'Бутырская':38.1293,'Нагатинская':38.2679,'Студенческая':38.2818,'Семёновская':38.321,'Аэропорт':38.455,'Академическая':38.5358,'Лефортово':38.6605,'Площадь трёх вокзалов':38.7806,'Коломенская':38.8222,'Каширская':38.9538,'Нагорная':39.1732,'Воробьёвы горы':39.1894,'Нижегородская':39.2448,'Петровско-Разумовская':39.254,'Народное Ополчение':39.3072,'Деловой центр':39.3072,'Печатники':39.3187,'ВДНХ':39.3418,'Кожуховская':39.3533,'Профсоюзная':39.3788,'Шелепиха':39.3834,'Каховская':39.4042,'Севастопольская':39.4296,'Славянский бульвар':39.4665,'Серп и Молот':39.5612,'Выставочная':39.5635,'Октябрьское Поле':39.5751,'Шоссе Энтузиастов':39.6628,'Нахимовский проспект':39.6905,'Казанский вокзал':39.7136,'Новые Черёмушки':39.746,'Сокол':39.9723,'Крымская':40.0531,'Фонвизинская':40.1178,'Воронцовская':40.2032,'Минская':40.2032,'Калужская':40.2794,'Деловой Центр':40.2864,'Кунцевская':40.3441,'Университет':40.3464,'Варшавская':40.4527,'Верхние Котлы':40.4804,'Мнёвники':40.545,'Проспект Вернадского':40.582,'Партизанская':40.6305,'Международная':40.6536,'Кузьминки':40.6998,'Хорошёво':40.9446,'Фили':40.9561,'Нагатинский Затон':40.9746,'Владыкино':40.9931,'Преображенская площадь':41.0393,'Зюзино':41.0393,'Ленинградский вокзал':41.2102,'Терехово':41.2356,'Кленовый бульвар':41.3279,'Зил':41.3811,'Мичуринский проспект':41.6074,'Новаторская':41.6282,'Войковская':41.7968,'Ботанический сад':41.8083,'Аминьевская':41.8245,'Угрешская':41.9053,'Новохохловская':41.9238,'Давыдково':41.9908,'Волжская':42.037,'Окружная':42.0393,'Кантемировская':42.0439,'Щукинская':42.1663,'Москва-Товарная':42.3418,'Зорге':42.3903,'Черкизовская':42.4642,'Беляево':42.5081,'Перово':42.5751,'Ломоносовский проспект':42.6351,'Чертановская':42.6905,'Измайлово':42.836,'Багратионовская':42.9307,'Панфиловская':43.03,'Рязанский проспект':43.1871,'Локомотив':43.1871,'Андроновка':43.3164,'Молодёжная':43.4688,'Соколиная Гора':43.6051,'Раменки':43.6351,'Филёвский парк':43.7206,'Измайловская':43.7229,'Пионерская':43.7506,'Стрешнево':43.9192,'Юго-Западная':43.963,'Отрадное':44.097,'Верхние Лихоборы':44.1824,'Сортировочная':44.2725,'Коньково':44.3811,'Калитники':44.3811,'Балтийская':44.3972,'Лихоборы':44.418,'Тестовская':44.4896,'Царицыно':44.6674,'Водный стадион':44.709,'Люблино':44.8776,'Свиблово':44.97,'Спартак':45.0162,'Ростокино':45.037,'Коптево':45.1455,'Бульвар Рокоссовского':45.2217,'Новогиреево':45.4018,'Южная':45.4988,'Белокаменная':45.6374,'Стахановская':45.709,'Чухлинка':45.8129,'Селигерская':46.0647,'Выхино':46.067,'Крылатское':46.1686,'Гражданская':46.2564,'Поклонная':46.5681,'Первомайская':46.6952,'Тропарёво':46.8661,'Бабушкинская':46.9515,'Останкино':47.0092,'Бибирево':47.0693,'Красный Балтиец':47.1871,'Тёплый Стан':47.194,'Пражская':47.3718,'Тушинская':47.4157,'Речной вокзал':47.4734,'Орехово':47.5427,'Братиславская':47.6028,'Матвеевская':48.1778,'Окская':48.2887,'Озёрная':48.4758,'Щёлковская':48.6859,'Яхромская':48.8915,'Кусково':49.1155,'Новокосино':49.2771,'Беломорская':49.3141,'Марьино':49.3972,'Лермонтовский проспект':49.4226,'Рабочий Посёлок':49.6952,'Медведково':49.9376,'Ясенево':50.0023,'Алтуфьево':50.0554,'Улица Академика Янгеля':50.1848,'Депо':50.2148,'Домодедовская':50.3303,'Сходненская':50.5543,'Плющево':50.6143,'Румянцево':50.7552,'Лианозово':50.7575,'Косино':50.8614,'Ховрино':51.0762,'Юго-Восточная':51.1732,'Дегунино':51.358,'Говорово':51.3788,'Красногвардейская':51.3857,'Очаково':51.6305,'Борисово':51.6374,'Новоясеневская':51.8453,'Аннино':52.0577,'Вешняки':52.1501,'Зябликово':52.1963,'Шипиловская':52.3764,'Жулебино':52.5473,'Сетунь':52.6259,'Строгино':52.6282,'Саларьево':52.709,'Перерва':52.8891,'Планерная':53.5404,'Бескудниково':54.1848,'Солнцево':54.2956,'Мещерская':54.4365,'Курьяново':54.5358,'Битцевский парк':54.6236,'Трикотажная':54.6282,'Бульвар Дмитрия Донского':54.7182,'Физтех':54.7621,'Улица Дмитриевского':54.8106,'Реутов':54.8199,'Моссельмаш':54.9746,'Мякинино':54.9838,'Москворечье':55.037,'Волоколамская':55.2656,'Котельники':55.5335,'Немчиновка':55.5704,'Алма-Атинская':56.6443,'Лухмановская':56.7921,'Улица Старокачаловская':56.9145,'Ухтомская':57.1293,'Боровское шоссе':57.2263,'Солнечная':57.2333,'Грачёвская':57.4434,'Филатов Луг':57.6166,'Лесопарковая':58.2979,'Митино':58.4781,'Сколково':58.5289,'Никольское':58.7737,'Новопеределкино':59.1894,'Пенягино':59.3764,'Некрасовка':59.7783,'Покровское':60.0808,'Переделкино':60.1178,'Марк':60.1293,'Пятницкое шоссе':60.4688,'Баковка':60.5104,'Прокшино':60.5751,'Левобережная':60.6859,'Рассказовка':61.1617,'Салтыковская':61.7182,'Люберцы':61.806,'Павшино':62.3072,'Красный Строитель':62.9838,'Мичуринец':62.9931,'Улица Скобелевская':63.1293,'Химки':63.5889,'Новодачная':64.0185,'Бульвар Адмирала Ушакова':64.1155,'Одинцово':64.4919,'Ольховая':64.5381,'Панки':64.6259,'Кучино':64.6767,'Битца':64.9284,'Пыхтино':65.1247,'Красногорская':65.2517,'Внуково':65.8822,'Долгопрудная':65.9723,'Улица Горчакова':66.097,'Томилино':66.515,'Коммунарка':66.5289,'Ольгино':67.649,'Бунинская Аллея':68.0878,'Бутово':68.836,'Водники':68.9169,'Аэропорт Внуково':69.1062,'Опалиха':69.1963,'Красково':69.3626,'Молжаниново':69.4226,'Лесной Городок':70.7206,'Хлебниково':70.8891,'Аникеевка':72.1686,'Малаховка':72.224,'Новоподрезково':72.3533,'Железнодорожная':72.6259,'Щербинка':72.7621,'Шереметьевская':73.8614,'Подрезково':74.3164,'Толстопальцево':74.6097,'Удельная':75.0993,'Нахабино':75.1547,'Остафьево':75.7206,'Быково':77.0254,'Сходня':77.2748,'Кокошкино':77.5404,'Лобня':77.843,'Силикатная':78.6928,'Ильинская':79.9284,'Санино':80.485,'Фирсановская':81.2379,'Подольск':82.6744,'Отдых':82.8453,'Крёкшино':83.4434,'Победа':85.4249,'Кратово':85.776,'Есенинская':87.739,'Апрелевка':89.4065,'Зеленоград-Крюково':90.1963,'Фабричная':90.6975,'Раменское':92.679,'Ипподром':94.6697}

station_places = {'Пушкинская':1,'Театральная':2,'Чеховская':3,'Тверская':4,'Кузнецкий Мост':5,'Третьяковская':6,'Арбатская':7,'Китай-город':8,'Боровицкая':9,'Площадь Революции':10,'Таганская':11,'Киевская':12,'Новокузнецкая':13,'Добрынинская':14,'Краснопресненская':15,'Октябрьская':16,'Лубянка':17,'Баррикадная':18,'Тургеневская':19,'Охотный Ряд':20,'Павелецкая':21,'Курская':22,'Парк культуры':23,'Библиотека имени Ленина':24,'Чистые пруды':25,'Цветной бульвар':26,'Сретенский бульвар':27,'Марксистская':28,'Белорусская':29,'Новослободская':30,'Смоленская':31,'Маяковская':32,'Менделеевская':33,'Александровский сад':34,'Трубная':35,'Серпуховская':36,'Комсомольская':37,'Проспект Мира':38,'Чкаловская':39,'Полянка':40,'Пролетарская':41,'Савёловская':42,'Шаболовская':43,'Сухаревская':44,'Красные Ворота':45,'Кропоткинская':46,'Площадь Ильича':47,'Римская':48,'Улица 1905 года':49,'Достоевская':50,'Марьина Роща':51,'Бауманская':52,'Ленинский проспект':53,'Рижская':54,'Динамо':55,'Дмитровская':56,'Беговая':57,'Петровский парк':58,'Полежаевская':59,'Крестьянская Застава':60,'Электрозаводская':61,'Хорошёвская':62,'Волгоградский проспект':63,'Автозаводская':64,'Тульская':65,'Парк Победы':66,'Фрунзенская':67,'Сокольники':68,'Красносельская':69,'Авиамоторная':70,'Площадь Гагарина':71,'Тимирязевская':72,'ЦСКА':73,'Спортивная':74,'Дубровка':75,'Алексеевская':76,'Текстильщики':77,'Лужники':78,'Кутузовская':79,'Технопарк':80,'Бутырская':81,'Нагатинская':82,'Студенческая':83,'Семёновская':84,'Аэропорт':85,'Академическая':86,'Лефортово':87,'Площадь трёх вокзалов':88,'Коломенская':89,'Каширская':90,'Нагорная':91,'Воробьёвы горы':92,'Нижегородская':93,'Петровско-Разумовская':94,'Народное Ополчение':95,'Деловой центр':96,'Печатники':97,'ВДНХ':98,'Кожуховская':99,'Профсоюзная':100,'Шелепиха':101,'Каховская':102,'Севастопольская':103,'Славянский бульвар':104,'Серп и Молот':105,'Выставочная':106,'Октябрьское Поле':107,'Шоссе Энтузиастов':108,'Нахимовский проспект':109,'Казанский вокзал':110,'Новые Черёмушки':111,'Сокол':112,'Крымская':113,'Фонвизинская':114,'Воронцовская':115,'Минская':116,'Калужская':117,'Деловой Центр':118,'Кунцевская':119,'Университет':120,'Варшавская':121,'Верхние Котлы':122,'Мнёвники':123,'Проспект Вернадского':124,'Партизанская':125,'Международная':126,'Кузьминки':127,'Хорошёво':128,'Фили':129,'Нагатинский Затон':130,'Владыкино':131,'Преображенская площадь':132,'Зюзино':133,'Ленинградский вокзал':134,'Терехово':135,'Кленовый бульвар':136,'Зил':137,'Мичуринский проспект':138,'Новаторская':139,'Войковская':140,'Ботанический сад':141,'Аминьевская':142,'Угрешская':143,'Новохохловская':144,'Давыдково':145,'Волжская':146,'Окружная':147,'Кантемировская':148,'Щукинская':149,'Москва-Товарная':150,'Зорге':151,'Черкизовская':152,'Беляево':153,'Перово':154,'Ломоносовский проспект':155,'Чертановская':156,'Измайлово':157,'Багратионовская':158,'Панфиловская':159,'Рязанский проспект':160,'Локомотив':161,'Андроновка':162,'Молодёжная':163,'Соколиная Гора':164,'Раменки':165,'Филёвский парк':166,'Измайловская':167,'Пионерская':168,'Стрешнево':169,'Юго-Западная':170,'Отрадное':171,'Верхние Лихоборы':172,'Сортировочная':173,'Коньково':174,'Калитники':175,'Балтийская':176,'Лихоборы':177,'Тестовская':178,'Царицыно':179,'Водный стадион':180,'Люблино':181,'Свиблово':182,'Спартак':183,'Ростокино':184,'Коптево':185,'Бульвар Рокоссовского':186,'Новогиреево':187,'Южная':188,'Белокаменная':189,'Стахановская':190,'Чухлинка':191,'Селигерская':192,'Выхино':193,'Крылатское':194,'Гражданская':195,'Поклонная':196,'Первомайская':197,'Тропарёво':198,'Бабушкинская':199,'Останкино':200,'Бибирево':201,'Красный Балтиец':202,'Тёплый Стан':203,'Пражская':204,'Тушинская':205,'Речной вокзал':206,'Орехово':207,'Братиславская':208,'Матвеевская':209,'Окская':210,'Озёрная':211,'Щёлковская':212,'Яхромская':213,'Кусково':214,'Новокосино':215,'Беломорская':216,'Марьино':217,'Лермонтовский проспект':218,'Рабочий Посёлок':219,'Медведково':220,'Ясенево':221,'Алтуфьево':222,'Улица Академика Янгеля':223,'Депо':224,'Домодедовская':225,'Сходненская':226,'Плющево':227,'Румянцево':228,'Лианозово':229,'Косино':230,'Ховрино':231,'Юго-Восточная':232,'Дегунино':233,'Говорово':234,'Красногвардейская':235,'Очаково':236,'Борисово':237,'Новоясеневская':238,'Аннино':239,'Вешняки':240,'Зябликово':241,'Шипиловская':242,'Жулебино':243,'Сетунь':244,'Строгино':245,'Саларьево':246,'Перерва':247,'Планерная':248,'Бескудниково':249,'Солнцево':250,'Мещерская':251,'Курьяново':252,'Битцевский парк':253,'Трикотажная':254,'Бульвар Дмитрия Донского':255,'Физтех':256,'Улица Дмитриевского':257,'Реутов':258,'Моссельмаш':259,'Мякинино':260,'Москворечье':261,'Волоколамская':262,'Котельники':263,'Немчиновка':264,'Алма-Атинская':265,'Лухмановская':266,'Улица Старокачаловская':267,'Ухтомская':268,'Боровское шоссе':269,'Солнечная':270,'Грачёвская':271,'Филатов Луг':272,'Лесопарковая':273,'Митино':274,'Сколково':275,'Никольское':276,'Новопеределкино':277,'Пенягино':278,'Некрасовка':279,'Покровское':280,'Переделкино':281,'Марк':282,'Пятницкое шоссе':283,'Баковка':284,'Прокшино':285,'Левобережная':286,'Рассказовка':287,'Салтыковская':288,'Люберцы':289,'Павшино':290,'Красный Строитель':291,'Мичуринец':292,'Улица Скобелевская':293,'Химки':294,'Новодачная':295,'Бульвар Адмирала Ушакова':296,'Одинцово':297,'Ольховая':298,'Панки':299,'Кучино':300,'Битца':301,'Пыхтино':302,'Красногорская':303,'Внуково':304,'Долгопрудная':305,'Улица Горчакова':306,'Томилино':307,'Коммунарка':308,'Ольгино':309,'Бунинская Аллея':310,'Бутово':311,'Водники':312,'Аэропорт Внуково':313,'Опалиха':314,'Красково':315,'Молжаниново':316,'Лесной Городок':317,'Хлебниково':318,'Аникеевка':319,'Малаховка':320,'Новоподрезково':321,'Железнодорожная':322,'Щербинка':323,'Шереметьевская':324,'Подрезково':325,'Толстопальцево':326,'Удельная':327,'Нахабино':328,'Остафьево':329,'Быково':330,'Сходня':331,'Кокошкино':332,'Лобня':333,'Силикатная':334,'Ильинская':335,'Санино':336,'Фирсановская':337,'Подольск':338,'Отдых':339,'Крёкшино':340,'Победа':341,'Кратово':342,'Есенинская':343,'Апрелевка':344,'Зеленоград-Крюково':345,'Фабричная':346,'Раменское':347,'Ипподром':348}

In [3]:
# utils

def poll_request(url: str):
    response = requests.get(url)
    time.sleep(first_sleep)
    while response.status_code == 500 or 'rate_limit' in response.text:
        if response.status_code == 500:
            print('got 500 status, trying more')
        elif 'rate_limit' in response.text:
            print('rate limited, trying more')
        time.sleep(sleep_interval)
        response = requests.get(url)
    return response

def write_to_file(file_path: str, text: str):
    with open(file_path, "w") as text_file:
        text_file.write(text)

def get_int_from_str(str: str):
    str = str.replace('\xa0', '')
    str = re.sub("[^0-9]", "", str)
    ans = 0
    for c in str:
        if c >= '0' and c <= '9':
            ans *= 10
            ans += (int)(c) - (int)('0')
    return ans

def create_url_from_params(min_cost, max_cost, room1, room2, room3, room4, limit, page):
    params = {
        'deal_type': 'rent',
        'engine_version': '2',
        'offer_type': 'flat',
        'region': '1',
        'type': '4'
    }
    if min_cost != 0:
        params['minprice'] = str(min_cost)
    if max_cost != 0:
        params['maxprice'] = str(max_cost)
    if min_area != 0:
        params['minarea'] = str(min_area)
    if max_area != 0:
        params['maxarea'] = str(max_area)
    if sort != '':
        params['sort'] = sort
    if room1 != 0:
        params['room1'] = str(room1)
    if room2 != 0:
        params['room2'] = str(room2)
    if room3 != 0:
        params['room3'] = str(room3)
    if room4 != 0:
        params['room4'] = str(room4)
    if limit != 0:
        params['limit'] = str(limit)
    if page != 0:
        params['p'] = str(page)

    url = 'https://www.cian.ru/cat.php?'
    for param_name, param_value in params.items():
        url += param_name + '=' + param_value + '&'
    while url.endswith('&'):
        url = url[:-1]
    return url

def underground_to_value(underground):
    ans = ''
    for station in underground.keys():
        ans += station + ': ' + str(underground[station]['time']) + ' мин. '
        if underground[station]['type'] == 'walk':
            ans += 'пешком; '
        else:
            ans += 'на транспорте; '
    return ans

def find_line_with_substring(multiline_str, substring):
    lines = multiline_str.splitlines()
    for line in lines:
        if substring in line:
            return line
    return None

def get_value(json, path, toString=True):
    for p in path:
        if p in json.keys():
            json = json[p]
        else:
            return 'UNDEF'
    if toString == False:
        return json
    if json == True:
        return 'Да'
    if json == False:
        return 'Нет'
    if json == 'euro':
        return 'Евро'
    if json == 'cosmetic':
        return 'Косметический'
    if json == 'design':
        return 'Дизайнерский'
    if json == 'yardAndStreet':
        return 'На улицу и двор'
    if json == 'yard':
        return 'Во двор'
    if json == 'street':
        return 'На улицу'
    return str(json)

def parse_undergrounds(json):
    values = {}
    for station in json:
        value = {}
        value['time'] = get_value(station, ['travelTime'], False)
        value['type'] = get_value(station, ['travelType'], False)
        values[get_value(station, ['name'])] = value
    return values

def write_as_text(cnt, values):
    ans = '\n' + 'Квартира № ' + str(cnt)
    for [key, value] in values.items():
        if key == 'underground_values':
            continue
        ans += '\n' + key + ': ' + str(value)
    ans += '\n\n\n'
    return ans

def get_values_from_object(href, objectResult):
    values = {
        'Ссылка': href
    }
    values['Цена'] = get_value(objectResult, ['value', 'offerData', 'offer', 'bargainTerms', 'price'])
    values['Месяцы предоплаты'] = get_value(objectResult, ['value', 'offerData', 'offer', 'bargainTerms', 'prepayMonths'], False)
    values['Депозит'] = get_value(objectResult, ['value', 'offerData', 'offer', 'bargainTerms', 'deposit'], False)
    values['Комиссия'] = get_value(objectResult, ['value', 'offerData', 'offer', 'bargainTerms', 'agentFee'], False)

    values['Апартаменты'] = get_value(objectResult, ['value', 'offerData', 'offer', 'isApartments'])
    values['Общая площадь'] = get_value(objectResult, ['value', 'offerData', 'offer', 'totalArea'], False)
    values['Жилая площадь'] = get_value(objectResult, ['value', 'offerData', 'offer', 'livingArea'], False)
    values['Площадь кухни'] = get_value(objectResult, ['value', 'offerData', 'offer', 'kitchenArea'], False)
    values['Этаж'] = get_value(objectResult, ['value', 'offerData', 'offer', 'floorNumber'], False)
    values['Обновлено'] = get_value(objectResult, ['value', 'offerData', 'offer', 'humanizedEditDate'])
    values['Лоджии'] = get_value(objectResult, ['value', 'offerData', 'offer', 'loggiasCount'], False)
    values['Балконы'] = get_value(objectResult, ['value', 'offerData', 'offer', 'balconiesCount'], False)
    values['Окна'] = get_value(objectResult, ['value', 'offerData', 'offer', 'windowsViewType'])
    values['Разделенные с/у'] = get_value(objectResult, ['value', 'offerData', 'offer', 'separateWcsCount'], False)
    values['Совмещенные с/у'] = get_value(objectResult, ['value', 'offerData', 'offer', 'combinedWcsCount'], False)
    values['Ремонт'] = get_value(objectResult, ['value', 'offerData', 'offer', 'repairType'])
    values['Посудомойка'] = get_value(objectResult, ['value', 'offerData', 'offer', 'hasWasher'])
    values['Кондиционер'] = get_value(objectResult, ['value', 'offerData', 'offer', 'hasConditioner'])
    values['Можно с детьми'] = get_value(objectResult, ['value', 'offerData', 'offer', 'childrenAllowed'])
    values['Можно с животными'] = get_value(objectResult, ['value', 'offerData', 'offer', 'petsAllowed'])
    values['Количество комнат'] = get_value(objectResult, ['value', 'offerData', 'offer', 'roomsCount'])

    values['Год постройки'] = get_value(objectResult, ['value', 'offerData', 'offer', 'building', 'buildYear'])
    if (values['Год постройки'] == 'UNDEF'):
        values['Год постройки'] = get_value(objectResult, ['value', 'offerData', 'bti', 'houseData', 'yearRelease'])

    values['Количество этажей'] = get_value(objectResult, ['value', 'offerData', 'offer', 'building', 'floorsCount'])
    if (values['Количество этажей'] == 'UNDEF'):
        values['Количество этажей'] = get_value(objectResult, ['value', 'offerData', 'bti', 'houseData', 'floorMax'])

    values['Высота потолков'] = get_value(objectResult, ['value', 'offerData', 'offer', 'building', 'ceilingHeight'])
    values['Количество подъездов'] = get_value(objectResult, ['value', 'offerData', 'bti', 'houseData', 'entrances'])
    values['Количество лифтов в доме'] = get_value(objectResult, ['value', 'offerData', 'bti', 'houseData', 'lifts'])
    values['Количество квартир в доме'] = get_value(objectResult, ['value', 'offerData', 'bti', 'houseData', 'flatCount'])


    underground_values = parse_undergrounds(get_value(objectResult, ['value', 'offerData', 'offer', 'geo', 'undergrounds'], False))
    values['underground_values'] = underground_values
    values['Близость к метро'] = underground_to_value(underground_values)
    return values


In [ ]:
# scoring

INF = 1000000000.0
DEFAULT_VALUE = 800000.0

def get_cost_value(cost, deposit):
    cost_value = get_int_from_str(cost)
    if cost_value == 0 or cost_value > max_cost:
        cost_value = get_int_from_str(deposit)
    return -cost_value

def get_deposit_value(deposit):
    cost_value = get_int_from_str(deposit)
    return -cost_value*0.02

def get_all_area_value(area):
    area_float = float(area)
    return area_float * 300.0

def get_live_area_value(area, all_area):
    coef = 1.0
    if (all_area == 'UNDEF'):
        return -INF
    if (area == '0.0' or area == 'UNDEF'):
        area = all_area
        coef = 0.5
    area_float = float(area)
    return area_float * coef * 600.0

def get_kitchen_area_value(area):
    if (area == '0.0' or area == 'UNDEF'):
        area_float = 5.0
    else:
        area_float = float(area)
    if area_float > 100.0:
        return -INF
    return area_float * 50.0


def get_comission_value(comission):
    if (comission == 'UNDEF'):
        return 0TO.0
    return -35 * comission

def get_premonth_value(months):
    if (get_premonth_value == 'UNDEF'):
        return 0.0TOFO
    return -3500 * months

def get_station_score(station):
    return station_scores.get(station, 100.0)

def get_station_place(station):
    return station_places.get(station, 1111)

def get_underground_score(stations):
    best = 1000.0
    top_place = 0
    for station in stations.keys():
        cur = get_station_score(station)
        if stations[station]['type'] == 'walk':
            cur += stations[station]['time'] * 1.1
        else:
            cur += stations[station]['time'] * 3.3
        if (cur < best):
            best = cur
            top_place = get_station_place(station)
        best = min(best, cur)
    result = best - (len(stations) - 1) * 2
    return [result, top_place]

def get_underground_value(stations):
    score = get_underground_score(stations)
    result = [(50.0 - score[0]) * 2000.0, score[0], score[1]]
    return result

def get_flat_score(values):
    underground = get_underground_value(values.get('underground_values', {}))
    return [(
        DEFAULT_VALUE +
        get_cost_value(values.get('Цена', 'UNDEF'), values.get('Залог', 'UNDEF')) +
        get_all_area_value(values.get('Общая площадь', '0.0')) +
        get_live_area_value(values.get('Жилая площадь', '0.0'), values.get('Общая площадь', '0.0')) +
        get_kitchen_area_value(values.get('Площадь кухни', '0.0')) +
        get_deposit_value(values.get('Залог', 'UNDEF')) +
        get_comission_value(values.get('Комиссия', '0.0')) +
        get_premonth_value(values.get('Месяцы предоплаты', '0.0')) +
        underground[0]
    ),
    underground[1],
    underground[2]
    ]

In [5]:
# main code

cnt = 0
result = ''
found = {}

for page in range(page_number):
    url = create_url_from_params(min_cost, max_cost, room1, room2, room3, room4, limit, page + 1)
    response = poll_request(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    apartments = soup.find_all('div', {'data-name': 'LinkArea'})

    for i in range (len(apartments)):
        if i % 2:
            continue
        href = apartments[i].find('a', {'class': '_93444fe79c--link--eoxce'})["href"]
        if href not in found.keys():
            found[href] = 'true'
            response = poll_request(href)

            substring = "window._cianConfig['frontend-offer-card']"
            prefix_str = "window._cianConfig['frontend-offer-card'] = (window._cianConfig['frontend-offer-card'] || []).concat("
            info = find_line_with_substring(response.text, substring)
            if info is not None:
                info = info[:-2]
            if info is not None:
                info = info[len(prefix_str):]

            jsonInfo = json.loads(info)
            objectResult = None
            for object in jsonInfo:
                if object['key'] == 'defaultState':
                    objectResult = object
            if (objectResult is None):
                print('object is None')
                continue

            values = get_values_from_object(href, objectResult)
            score = get_flat_score(values)
            values['Flat Score'] = str(round(score[0]))
            values['Underground Score'] = str(round(score[1], 2))
            values['Underground Place'] = str(score[2])

            print(values)

            cnt += 1
            if cnt % 5 == 0:
                print('done: ' + str(cnt) + ' / ' + str(min(len(apartments) // 2 * page_number, overall_limit)))

            result += write_as_text(cnt, values)
            for key in lists.keys():
                lists[key].append(values.get(key, ''))

            if cnt >= overall_limit:
                break

    write_to_file(MAIN_DIR + "/parsing_result.txt", result)
    dataframe = pd.DataFrame(lists)
    dataframe.to_csv(MAIN_DIR + "/report_table.csv")

    print(f'text report saved as {MAIN_DIR}parsing_report.txt')
    print(f'CSV report saved as {MAIN_DIR}report_table.csv')

{'Ссылка': 'https://www.cian.ru/rent/flat/311384887/', 'Цена': '99000', 'Месяцы предоплаты': 1, 'Депозит': 99000, 'Комиссия': 80, 'Апартаменты': 'Нет', 'Общая площадь': '95.0', 'Жилая площадь': '50.0', 'Площадь кухни': '12.0', 'Этаж': 15, 'Обновлено': 'сегодня, 17:05', 'Лоджии': 2, 'Балконы': 0, 'Окна': 'На улицу и двор', 'Разделенные с/у': 1, 'Совмещенные с/у': 2, 'Ремонт': 'Дизайнерский', 'Посудомойка': 'Да', 'Кондиционер': 'Нет', 'Можно с детьми': 'UNDEF', 'Можно с животными': 'UNDEF', 'Количество комнат': '3', 'Год постройки': '1982', 'Количество этажей': '16', 'Высота потолков': '2.7', 'Количество подъездов': '3', 'Количество лифтов в доме': '10', 'Количество квартир в доме': '123', 'underground_values': {'Чертановская': {'time': 11, 'type': 'walk'}, 'Севастопольская': {'time': 19, 'type': 'walk'}, 'Каховская': {'time': 22, 'type': 'walk'}}, 'Близость к метро': 'Чертановская: 11 мин. пешком; Севастопольская: 19 мин. пешком; Каховская: 22 мин. пешком; ', 'Flat Score': '752219', 'Un